In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
def Nurse_2_Nurse_Summarization_v1(filepath: str):

    llm = ChatOpenAI(model = 'gpt-4o')
    
    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs  = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 1000, add_start_index = True)
    all_splits = text_splitter.split_documents(docs)

    embed = OpenAIEmbeddings(show_progress_bar=True, request_timeout= 10000)

    vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed, persist_directory= './gurshan_3_db')
    retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={'k': 6})

    template = """Use the following pieces of context to summarize the data at the end.
    If you dont know, just say you dont know. 
    Pretend you are a health care worker at the ned of your shift. 
    Now is the time for patient handover. Explain and summarize to the next nurse the status of the patient described in the JSON file.
    In the case of irregular vital sign fluctuations, flag it and mention it 

    Always say "The End!", in a new line at the end of the answer.

    {context}

    Question: {question}

    Helpful Answer:"""
    custom_rag_prompt = PromptTemplate.from_template(template)
    rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | custom_rag_prompt | llm | StrOutputParser())
    response = rag_chain.invoke("Summarize")

    return response

    

In [5]:
Nurse_2_Nurse_Summarization_v1('Gurshan_Data.json')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

"The patient has experienced several fluctuations in vital signs throughout the shift, with notable irregularities in heart rate and respiratory rate. The heart rate has ranged from 130 to 158 bpm, and the respiratory rate has varied from 34 to 47 breaths per minute. The patient's skin tone has alternated between normal, red, blueish, pink, and yellow-jaundice, indicating potential underlying issues. There have been multiple interventions, including lighting adjustments, oxygen administration, feeding, diaper changes, medication, and position changes, to address varying pain levels ranging from low to extreme.\n\nSignificant irregularities include periods of elevated heart rate and respiratory rate, particularly when the skin tone was blueish or red, suggesting respiratory distress or inadequate oxygenation at times. Pain levels have also fluctuated significantly, affecting the patient's comfort and requiring frequent analgesic and antibiotic administration.\n\nThe patient requires clo